In [1]:
from dgp import *

# 1. Simulate or load data
sample_size = 800
min_degree = 1
max_degree = 1
adj = get_graph(sample_size, min_degree, max_degree, seed=2)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=30,
    burnin_R=10, seed=1)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:01<00:00, 30.78it/s]


In [2]:
from drnet import doubly_robust
from run_pll import cols_raw

ret_i = doubly_robust(A_chain[-2], L_chain[-2], Y_chain[-2], adj, return_raw=True)
ret_array = np.zeros((ret_i[cols_raw[0]].shape[0], ret_i[cols_raw[0]].shape[1], len(cols_raw)))
for i in range(len(cols_raw)):
    ret_array[:, :, i] = ret_i[cols_raw[i]].copy()
print(ret_array.shape)

(800, 1000, 4)


In [2]:
from autognet import fit_autog_models, extract_parameters_from_autog_models, prepare_features_outcome_model
import statsmodels.api as sm

def get_beta(adj_matrix, Y, A, L, treatment_allocation=0.5, R=10, burnin=5, seed=0):
    """
    Fit autognet models and evaluate causal effects using agc_effect.
    """
    np.random.seed(seed)
    models = fit_autog_models(Y, A, L, adj_matrix)
    tau, rho, nu, beta = extract_parameters_from_autog_models(models, adj_matrix)

    return beta

In [3]:
from tqdm import tqdm

betahats = []
for i in tqdm(range(len(Y_chain))):
    betahat = get_beta(adj, Y_chain[i], A_chain[i], L_chain[i], treatment_allocation=0.7, R=50, burnin=10, seed=1)
    betahats.append(betahat)

100%|██████████| 1000/1000 [00:17<00:00, 56.53it/s]


In [4]:
betahats[-1]

array([-0.39173937, -0.52787214,  0.15045679,  0.00440145, -0.08004167,
       -0.40050773, -0.05185024,  0.63315126,  0.14825143, -0.201992  ])

In [5]:
beta

array([-0.3 , -0.6 , -0.2 , -0.2 , -0.05, -0.1 , -0.01,  0.4 ,  0.01,
        0.2 ])

In [6]:
np.mean(np.array(betahats), axis=0) - beta

array([ 0.08010531, -0.0145634 , -0.00379152,  0.00883194, -0.00116777,
        0.00224138, -0.00691273,  0.00442347, -0.00247561, -0.21096924])

In [7]:
np.mean(np.mean(np.array(betahats), axis=0) - beta)

-0.014427816936012338

In [7]:
num_sample = 30
num_burn = 10
sample_size = 800

from dgp import *

# 1. Simulate or load data
min_degree = 2
max_degree = 4
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 40/40 [00:16<00:00,  2.46it/s]


In [2]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=20,
    burnin_R=5,
    seed=0
)

{'average': 0.6428125,
 'direct_effect': -0.1087499999999999,
 'spillover_effect': -0.08543750000000006,
 'psi_1_gamma': 0.6073125000000001,
 'psi_0_gamma': 0.7160625,
 'psi_zero': 0.8015000000000001}

In [ ]:
n_cpu = 10

In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

array([ 0.79701842, -0.08244906, -0.096901  ,  0.85651992,  0.95342093,
        0.77407086])

In [ ]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [ ]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

array([ 0.79645 , -0.029375, -0.099175,  0.8145  ,  0.913675,  0.785125])

In [ ]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")